# Phase 1: Data Ingestion Pipeline

Preprocesses raw IVF clinical data into two data stores for a RAG system:

| Step | What | Output |
|------|------|--------|
| 1 | **Anonymize** identifiers (SHA-256 hash) | Anonymized DataFrames + mapping file |
| 2 | **Load structured data** into SQLite | `clinic.db` with 4 tables + schema descriptions |
| 3 | **Process clinical notes** — strip HTML, build single document, chunk | Text chunks ready for embedding |
| 4 | **Embed and store** in ChromaDB | Vector index (setup only — run when ready) |

> **Note:** This notebook only runs preprocessing (Steps 1-3). Step 4 embedding code is provided but not auto-executed.

In [ ]:
import pandas as pd
import hashlib
import json
import re
import os
import sqlite3
import numpy as np
from pathlib import Path
from bs4 import BeautifulSoup
from html import unescape

In [ ]:
# ── Configuration ──
DATA_DIR = Path(".")  # Folder containing the 5 CSVs

# Chunking settings
CHUNK_SIZE = 300    # ~300 tokens per chunk
CHUNK_OVERLAP = 50  # 50-token overlap between consecutive chunks

# Identifier columns to hash (adapted from anonymize_data.py)
IDENTIFIER_COLUMNS = [
    "Patient ID",   # Follow-up notes, Reproduction Episodes, History Visits
    "Partner ID",   # Reproduction Episodes
    "Patient",      # Spermiogram
    "Partner",      # Spermiogram
]

# SQLite output
DB_PATH = "clinic.db"

## Step 1 — Anonymize Identifiers

Adapted from `anonymize_data.py` which hashes fields in JSON/JSONL files using SHA-256.
Here we apply the same hashing to identifier columns in pandas DataFrames loaded from CSV.
The mapping is saved to `anonymization_mapping.json` for traceability.

In [ ]:
def hash_value(value):
    """
    Hash a value using SHA-256.
    Identical logic to anonymize_data.py hash_value(), adapted for DataFrames.
    """
    if pd.isna(value) or value is None:
        return value
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()


def anonymize_dataframes(data_dict, id_columns):
    """
    Apply SHA-256 hashing to identifier columns across all DataFrames.

    Adapted from anonymize_data.py (which processes JSON/JSONL files with
    anonymize_dict()) to work on pandas DataFrames from CSV.

    Args:
        data_dict: dict of {table_name: DataFrame}
        id_columns: list of column names to hash

    Returns:
        data_dict with identifier columns hashed in-place
        mapping: {hashed_value: original_value} for traceability
    """
    mapping = {}

    for table_name, df in data_dict.items():
        for col in id_columns:
            if col not in df.columns:
                continue
            # Build mapping before overwriting
            for val in df[col].dropna().unique():
                hashed = hash_value(val)
                if hashed not in mapping:
                    mapping[hashed] = str(val)
            # Replace with hashes
            df[col] = df[col].apply(hash_value)
            print(f"  Hashed '{col}' in {table_name} ({df[col].notna().sum()} values)")

    return data_dict, mapping

In [ ]:
def strip_html(text):
    """
    Strip HTML tags from clinical notes and clean up whitespace.
    Clinical data has embedded tags like <br/>, <strong>, <p>, etc.
    """
    if pd.isna(text):
        return ""
    text = unescape(str(text))  # &amp; -> &, &#8217; -> ', etc.
    soup = BeautifulSoup(text, "html.parser")
    clean = soup.get_text(separator=" ", strip=True)
    return re.sub(r'\s+', ' ', clean).strip()

In [ ]:
def load_all_tables(data_dir):
    """Load all 5 CSVs and parse date columns."""
    cycles = pd.read_csv(data_dir / "Cycles.csv", encoding="utf-8-sig")
    cycles["Date"] = pd.to_datetime(cycles["Date"], dayfirst=True, errors="coerce")

    repro = pd.read_csv(data_dir / "Reproduction Episodes.csv", encoding="utf-8-sig")
    repro["Date"] = pd.to_datetime(repro["Date"], dayfirst=True, errors="coerce")

    notes = pd.read_csv(data_dir / "Follow up notes.csv", encoding="utf-8-sig")
    notes["Date"] = pd.to_datetime(notes["Date"], dayfirst=True, errors="coerce")
    notes = notes.sort_values("Date")

    sperm = pd.read_csv(data_dir / "Spermiogram.csv", encoding="utf-8-sig")
    sperm["Date"] = pd.to_datetime(sperm["Date"], dayfirst=True, errors="coerce")

    visits = pd.read_csv(data_dir / "History visits.csv", encoding="utf-8-sig")
    visits["Date"] = pd.to_datetime(visits["Date"], dayfirst=True, errors="coerce")
    visits = visits.sort_values("Date")

    return {
        "cycles": cycles,
        "reproduction_episodes": repro,
        "follow_up_notes": notes,
        "spermiogram": sperm,
        "history_visits": visits,
    }


# Load tables
data = load_all_tables(DATA_DIR)
for name, df in data.items():
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} columns")

In [ ]:
# Anonymize identifier columns
print("Anonymizing identifiers...")
data, id_mapping = anonymize_dataframes(data, IDENTIFIER_COLUMNS)

# Save mapping for traceability (same structure as anonymize_data.py output)
mapping_output = {
    "hash_to_original": id_mapping,
    "original_to_hash": {v: k for k, v in id_mapping.items()},
    "total_mappings": len(id_mapping),
}
with open("anonymization_mapping.json", "w") as f:
    json.dump(mapping_output, f, indent=2)

print(f"\nSaved {len(id_mapping)} identifier mapping(s) to anonymization_mapping.json")

In [ ]:
# Strip HTML from text-heavy columns
for col in ["Therapeutic Plan", "Cause of the episode", "Obstetrics observations"]:
    if col in data["reproduction_episodes"].columns:
        data["reproduction_episodes"][col] = data["reproduction_episodes"][col].apply(strip_html)

data["follow_up_notes"]["Follow-up_clean"] = data["follow_up_notes"]["Follow-up"].apply(strip_html)
data["spermiogram"]["Remarks_clean"] = data["spermiogram"]["Remarks"].apply(strip_html)

print("HTML stripped from clinical text columns.")
print(f"  Follow-up notes: {data['follow_up_notes']['Follow-up_clean'].str.len().mean():.0f} avg chars per note")

## Step 2 — Load Structured Data into SQLite

Takes the anonymized Cycles, History Visits, Spermiogram, and Reproduction Episodes
and loads them as SQL tables using `pandas.to_sql()`. A schema description dict tells
the LLM what each column means (e.g., `#COC` = oocytes retrieved, `MII` = mature oocytes).

In [ ]:
def create_sqlite_database(data, db_path):
    """Load structured DataFrames into a SQLite database."""
    if os.path.exists(db_path):
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    structured_tables = ["cycles", "reproduction_episodes", "spermiogram", "history_visits"]

    for table_name in structured_tables:
        df = data[table_name].copy()
        # Clean column names for SQL compatibility
        df.columns = [
            re.sub(r'[^a-zA-Z0-9_]', '_', col.strip()).strip('_')
            for col in df.columns
        ]
        df = df.dropna(axis=1, how='all')
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"  {table_name}: {len(df)} rows, {len(df.columns)} columns")

    print(f"\nDatabase saved to: {db_path}")
    return conn


# Schema descriptions — critical for the Text-to-SQL engine
# The LLM reads these to understand domain-specific column names
SCHEMA_DESCRIPTION = {
    "cycles": {
        "table_description": "IVF treatment cycles with outcomes and embryology results",
        "columns": {
            "Id": "Unique cycle identifier",
            "Process": "Type of procedure (e.g., I.V.F Stimulation, Endometrial Prep.)",
            "Protocol": "Stimulation protocol used (e.g., Antagonist fixed D5)",
            "Date": "Date the cycle started",
            "MD": "Treating physician initials",
            "Technique": "Lab technique (e.g., ICSI, VITRIFICATION)",
            "Ooc__Source": "Source of oocytes (Fresh Own, Combined, etc.)",
            "Semen_source": "Source of sperm (Fresh ejaculation, Frozen ejaculation, etc.)",
            "Outcome": "Cycle result (Pregnancy: Yes, No transfer: Embryo vitrification, etc.)",
            "Specific_Outcome": "Detailed outcome information",
            "Live_birth": "Whether a live birth resulted",
            "PGT": "Pre-implantation genetic testing type (PGT-A = aneuploidy screening)",
            "Ref__No_": "Reference number for the cycle",
            "_ET": "Number of embryos transferred",
            "_COC": "Number of cumulus-oocyte complexes (oocytes) retrieved",
            "MII": "Number of metaphase II (mature) oocytes",
            "_2PN": "Number of normally fertilized oocytes (2 pronuclei)",
            "Frozen_oocytes": "Number of oocytes frozen (vitrified)",
            "Frozen_embryo": "Number of embryos frozen",
            "Transferable_Emb_": "Number of embryos suitable for transfer",
            "Age": "Patient age at time of cycle",
            "BMI": "Patient BMI at time of cycle",
        },
    },
    "reproduction_episodes": {
        "table_description": "Patient reproductive history, demographics, and episode information",
        "columns": {
            "Id": "Episode identifier",
            "Date": "Episode start date",
            "Patient_ID": "Anonymized patient identifier (SHA-256 hash)",
            "Gestations": "Number of previous pregnancies",
            "Para": "Number of births",
            "Misscarriage": "Number of miscarriages",
            "E_P": "Number of ectopic pregnancies",
            "C_Sections": "Number of cesarean sections",
            "Preterm_Birth": "Number of preterm births",
            "Partner_ID": "Anonymized partner identifier (SHA-256 hash)",
            "Reason_Consultation": "Why the patient consulted (e.g., Primary infertility)",
            "Therapeutic_Plan": "Planned treatment approach (cleaned text)",
            "Cause_of_the_episode": "Clinical cause/diagnosis for this episode (cleaned text)",
            "Years_of_marriage": "Duration of marriage in years",
            "Nationality": "Patient nationality",
            "Cycle_length": "Menstrual cycle length in days",
            "Previous_treatments": "Number of previous fertility treatments",
        },
    },
    "spermiogram": {
        "table_description": "Semen analysis results for the male partner",
        "columns": {
            "Id": "Spermiogram test identifier",
            "Date": "Date the test was performed",
            "Patient": "Male partner identifier (SHA-256 hash)",
            "Partner": "Female partner identifier (SHA-256 hash)",
            "Volume____1_4_ml_": "Semen volume in ml (normal >= 1.4 ml)",
            "Count____16_M_ml_": "Sperm count in millions per ml (normal >= 16 M/ml)",
            "Progressive____30__": "Progressive motility percentage (normal >= 30%)",
            "Normal_Forms____4__": "Normal morphology percentage (normal >= 4%)",
            "Sperm_Motility_Recovery": "Total motile sperm after processing",
            "Diagnosis": "Spermiogram diagnosis (e.g., Normozoospermia, Oligozoospermia)",
            "Remarks": "Additional notes about the sample",
        },
    },
    "history_visits": {
        "table_description": "Patient visit history with anthropometric measurements over time",
        "columns": {
            "Code": "Visit identifier",
            "Patient_ID": "Anonymized patient identifier (SHA-256 hash)",
            "Date": "Visit date",
            "Age": "Patient age at visit",
            "Height": "Patient height in meters",
            "Weight": "Patient weight in kg",
            "BMI": "Body Mass Index (kg/m^2)",
        },
    },
}

# Save schema for the LLM to reference
with open("schema_description.json", "w") as f:
    json.dump(SCHEMA_DESCRIPTION, f, indent=2)
print("Schema descriptions saved to schema_description.json")

In [ ]:
# Create the SQLite database
conn = create_sqlite_database(data, DB_PATH)

# Verify
print("\nVerification:")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
for (table,) in cursor.fetchall():
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    cursor.execute(f"PRAGMA table_info({table})")
    cols = [row[1] for row in cursor.fetchall()]
    print(f"  {table}: {count} rows, {len(cols)} columns")

# Sample queries
print("\nSample: SELECT Date, Technique, Outcome FROM cycles LIMIT 3")
for row in cursor.execute("SELECT Date, Technique, Outcome FROM cycles LIMIT 3"):
    print(f"  {row}")

print("\nSample: SELECT Date, Age, BMI FROM history_visits LIMIT 3")
for row in cursor.execute("SELECT Date, Age, BMI FROM history_visits ORDER BY Date LIMIT 3"):
    print(f"  {row}")

## Step 3 — Process Clinical Notes for Vector Search

Combines all clinical free-text (follow-up notes, episode text fields, spermiogram remarks)
into **one document per patient** after stripping HTML. Then splits into ~300-token chunks
with 50-token overlap.

Each note is a natural boundary — only notes longer than 300 tokens get sub-chunked.

In [ ]:
def build_patient_document(data):
    """
    Combine all clinical free-text into a single document.
    Sources: follow-up notes, reproduction episode text fields, spermiogram remarks.
    All HTML has already been stripped in Step 1.
    """
    sections = []

    # Follow-up notes (bulk of unstructured text)
    notes_df = data["follow_up_notes"].sort_values("Date")
    for _, row in notes_df.iterrows():
        text = row.get("Follow-up_clean", "")
        if not text or len(str(text).strip()) < 10:
            continue
        date = str(row.get("Date", ""))[:10]
        note_type = str(row.get("Type", ""))
        sections.append(f"[Follow-up | {date} | {note_type}] {text}")

    # Reproduction episode free-text fields
    repro_df = data["reproduction_episodes"]
    for _, row in repro_df.iterrows():
        date = str(row.get("Date", ""))[:10]
        for col in ["Cause of the episode", "Therapeutic Plan", "Obstetrics observations"]:
            if col in repro_df.columns:
                text = str(row.get(col, ""))
                if text and len(text.strip()) > 10:
                    sections.append(f"[{col} | {date}] {text}")

    # Spermiogram remarks
    for _, row in data["spermiogram"].iterrows():
        text = row.get("Remarks_clean", "")
        if text and len(str(text).strip()) > 10:
            date = str(row.get("Date", ""))[:10]
            sections.append(f"[Spermiogram Remarks | {date}] {text}")

    return "\n\n".join(sections)


def chunk_document(document, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    """
    Split document into overlapping chunks of ~chunk_size tokens.
    Uses word count as proxy (1 token ~ 0.75 words).

    Each note (separated by double newline) is a natural boundary.
    Only notes longer than chunk_size tokens get sub-chunked.
    """
    notes = document.split("\n\n")
    chunks = []
    max_words = int(chunk_size * 0.75)
    overlap_words = int(overlap * 0.75)

    for note in notes:
        words = note.split()
        if not words:
            continue

        if len(words) <= max_words:
            # Short note — keep as one chunk
            chunks.append(note)
        else:
            # Long note — sub-chunk with sliding window overlap
            start = 0
            while start < len(words):
                end = start + max_words
                chunks.append(" ".join(words[start:end]))
                start = end - overlap_words

    return chunks

In [ ]:
# Build the single patient document
patient_document = build_patient_document(data)
print(f"Patient document: {len(patient_document):,} characters, {len(patient_document.split()):,} words")

# Chunk it
chunks = chunk_document(patient_document)
print(f"Total chunks: {len(chunks)}")

# Chunk size distribution
word_counts = [len(c.split()) for c in chunks]
print(f"Words per chunk: min={min(word_counts)}, max={max(word_counts)}, avg={np.mean(word_counts):.0f}")
print(f"Approx tokens:  min~{min(word_counts)/0.75:.0f}, max~{max(word_counts)/0.75:.0f}, avg~{np.mean(word_counts)/0.75:.0f}")

# Preview
print(f"\n--- Chunk 0 (first 300 chars) ---")
print(chunks[0][:300])
print(f"\n--- Chunk 1 (first 300 chars) ---")
print(chunks[1][:300])

## Step 4 — Embed and Store (Setup)

Embeds each text chunk using `BAAI/bge-large-en-v1.5` (1024-dim vectors) and stores
them in ChromaDB with the original text and metadata.

**Run the cell below when ready.** Requires:
```
pip install chromadb sentence-transformers
```

In [ ]:
# ── Step 4: Embed and Store ──
# Uncomment the block below and run when dependencies are installed.

# import chromadb
# from sentence_transformers import SentenceTransformer
#
# # Load embedding model (1024-dim vectors)
# embed_model = SentenceTransformer("BAAI/bge-large-en-v1.5")
#
# # Initialize ChromaDB (persistent local storage)
# chroma_client = chromadb.PersistentClient(path="./chroma_db")
# collection = chroma_client.get_or_create_collection(
#     name="clinical_notes",
#     metadata={"hnsw:space": "cosine"},
# )
#
# # Embed and store all chunks
# print(f"Embedding {len(chunks)} chunks...")
# for i, chunk_text in enumerate(chunks):
#     embedding = embed_model.encode(chunk_text, normalize_embeddings=True).tolist()
#     collection.add(
#         ids=[f"chunk_{i}"],
#         embeddings=[embedding],
#         documents=[chunk_text],
#         metadatas=[{"chunk_index": i}],
#     )
#     if (i + 1) % 50 == 0:
#         print(f"  Embedded {i + 1}/{len(chunks)} chunks...")
#
# print(f"\nStored {collection.count()} chunks in ChromaDB (./chroma_db)")
# print("Vector index ready for semantic search.")

print("Step 4 code is ready. Uncomment and run when chromadb + sentence-transformers are installed.")

## Summary

After running Steps 1-3, you have:

| Data Store | Contents | File |
|------------|----------|------|
| **SQLite** | 4 structured tables (cycles, reproduction_episodes, spermiogram, history_visits) | `clinic.db` |
| **Text chunks** | Clinical notes ready for embedding | `chunks` list in memory |
| **Schema** | Column descriptions for LLM text-to-SQL | `schema_description.json` |
| **ID mapping** | SHA-256 hash <-> original identifier | `anonymization_mapping.json` |

After running Step 4, you also have:

| Data Store | Contents | File |
|------------|----------|------|
| **ChromaDB** | Embedded note chunks (1024-dim vectors) | `./chroma_db/` |

These two data stores (SQLite + ChromaDB) are used side-by-side in Phase 2 (Query Pipeline).